<a href="https://colab.research.google.com/github/vgomesfalcao/planilha_central/blob/main/Capital_planilha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
pd.set_option('max_colwidth', None)

In [ ]:
sagecom = pd.read_excel('Planilha_Excel_Sygecom.xlsx')

In [7]:
razao = pd.read_excel('livro_razao_convertido.xlsx')
razao = razao.drop(range(8))
razao = razao.drop(razao.index[-1])
razao = razao.dropna(how='all', axis=1)
razao.columns = ['Data', 'Lançamento','Contra-Partida','Parenteses','Atalho','Histórico','Valor de Débito','Valor de Crédito', 'Saldo','Deb/Cred']
razao['Atalho'] = razao['Parenteses'] + razao['Atalho']
razao = razao.drop(columns=['Parenteses'])
razao = razao.reset_index(drop=True)

In [8]:
for row in razao.itertuples():
  if(pd.isnull(row.Data)):
    # display(row.Index-1)
    # break
    linha_anterior = razao.loc[[row.Index-1],'Histórico'].values[0]
    # print(linha_anterior)
    # break
    razao.loc[[row.Index-1],'Histórico'] =  linha_anterior + row.Histórico
    razao = razao.drop(row.Index)
    
  

KeyError: ignored

In [5]:
display(razao.iloc[[10,11,12,13,14,15]],)

,Data,Lançamento,Contra-Partida,Atalho,Histórico,Valor de Débito,Valor de Crédito,Saldo,Deb/Cred
10,01/04/2021,37484.0,2.111010e+09,(2429),ADT-FOR CH: 39051,NaN,1671.20,166822.34,D
11,01/04/2021,37495.0,1.112010e+09,(964),Transfere,300000.0,NaN,466822.34,D
12,01/04/2021,37506.0,2.111011e+09,(2501),"B-52096-1, B-53034-1, B-53511-1, B-53532-1, B-53600-1, B-",NaN,2934.75,463887.59,D
13,NaN,NaN,NaN,NaN,"54369-1, B-55372-1, B-56194-1, B-56254-1, B-56619-1, B-",NaN,NaN,NaN,NaN
14,NaN,NaN,NaN,NaN,56641-1 CH:39081,NaN,NaN,NaN,NaN
15,01/04/2021,37507.0,2.111011e+09,(2501),"B-52096-1, B-53034-1, B-53511-1, B-53532-1, B-53600-1, B-",NaN,2934.75,460952.84,D
